In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch

In [5]:
data = pd.read_csv('Video_views/youtube_train.csv')
data = data.drop(columns=['id', 'Channel'])
data = data.fillna('')
print(data.shape)
data.head()

(5849, 4)


,Subtitles,PublishDate,Category,ViewCount
0,('Я миллениал и я не занимаюсь сексом так част...,2017-12-31T01:35:44-08:00,Education,3019
1,"(""сервитуты это одно из прямых наследие римско...",2014-09-04T08:24:47-07:00,Science & Technology,25
2,('[музыка] существуют сенсоры измеряющие магни...,2023-11-23T21:00:08-08:00,Science & Technology,7
3,('в наше время только ленивые не говорит о том...,2014-06-23T03:49:31-07:00,Science & Technology,5
4,('дорогие друзья это честный рекламный ролик н...,2020-10-09T01:30:04-07:00,Education,10


In [6]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('DeepPavlov/rubert-base-cased')
model = BertModel.from_pretrained("DeepPavlov/rubert-base-cased")
# custom_text = "You are welcome to utilize any text of your choice."
# encoded_input = tokenizer(custom_text, return_tensors='pt', padding=True, truncation=True, max_length=512)
# output_embeddings = model(**encoded_input)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
# embedding = model(**encoded_input).last_hidden_state.cpu().detach().squeeze().numpy()
# embedding.sum(axis=0).shape

In [9]:
# model = BertModel.from_pretrained("Databook/db-TinyBERT")
# model.to(torch.device('cuda'))
# tokenizer = BertTokenizer.from_pretrained('Databook/db-TinyBERT')

# model = BertModel.from_pretrained("bert-base-cased")
model.to(torch.device('cuda'))
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')


def bert_embeddings(data: pd.DataFrame, col: str):
    embeddings = []
    tokenized_texts = tokenizer(data[col].to_list(), return_tensors='pt', padding=True, truncation=True, max_length=512)
    for i in tqdm(range(tokenized_texts['input_ids'].shape[0])):
        tokenized = {key: val[[i]].to(torch.device('cuda')) for key, val in tokenized_texts.items()}
        embedding = model(**tokenized).last_hidden_state.cpu().detach().squeeze().numpy()
        embedding = embedding.sum(axis=0)
        embeddings.append(embedding)
    return np.array(embeddings)

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

def vectorize_col(data: pd.DataFrame, col: str, vectorizer=None):
    texts_samples = data[col].to_list()
    if not vectorizer:
        vectorizer = TfidfVectorizer(stop_words='english', max_features=800, ngram_range=(1, 2))
    tf_idf = vectorizer.fit_transform(texts_samples)
    return tf_idf, vectorizer


In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


text = ['Subtitles']
categorical = ['Category']
# numeric = ['num_prev_rounds', 'ln_raised_amount', 'participants', 'ipo_prob', 'ma_prob']
# other = ['has_raised_amount', 'previous_any_founder_experience']

In [12]:
vectors_list = []
for col in text:
    vectorized_texts = bert_embeddings(data, col)
    vectors_list.append(vectorized_texts)

column_transformer = ColumnTransformer([
    ('ohe', OneHotEncoder(handle_unknown='ignore'), categorical),
    # ('scaling', StandardScaler(), numeric),
    # ('other',  'passthrough', other)
])

target = data['ViewCount'].to_numpy()
features = column_transformer.fit_transform(data).toarray()
features = np.concatenate([features, *vectors_list], axis=1)

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, train_size=0.8, random_state=3)

In [26]:
from sklearn.metrics import accuracy_score

def calc_score(true, pred):
    return 32 * (accuracy_score(true, pred) - 0.5)

In [27]:
from catboost import CatBoostClassifier

catboost_features = {
    'iterations': 1000,
    'random_seed': 1,
    'eval_metric': 'Accuracy',
    'verbose': 100,
    # 'task_type': 'GPU',
}
model = CatBoostClassifier(
    **catboost_features
)
model.fit(X_train, y_train, eval_set=(X_test, y_test))
pred = model.predict(X_test)
calc_score(y_test, pred)

Learning rate set to 0.045715
0:	learn: 0.6048991	test: 0.5793291	best: 0.5793291 (0)	total: 321ms	remaining: 5m 21s
100:	learn: 0.7827172	test: 0.6137806	best: 0.6264733 (11)	total: 21.9s	remaining: 3m 14s
200:	learn: 0.8772964	test: 0.6137806	best: 0.6264733 (11)	total: 43.4s	remaining: 2m 52s
300:	learn: 0.9614425	test: 0.5965549	best: 0.6264733 (11)	total: 1m 4s	remaining: 2m 30s
400:	learn: 0.9879791	test: 0.5947416	best: 0.6264733 (11)	total: 1m 26s	remaining: 2m 9s
500:	learn: 0.9984123	test: 0.6029012	best: 0.6264733 (11)	total: 1m 48s	remaining: 1m 47s
600:	learn: 0.9995464	test: 0.5965549	best: 0.6264733 (11)	total: 2m 9s	remaining: 1m 26s
700:	learn: 1.0000000	test: 0.5865820	best: 0.6264733 (11)	total: 2m 31s	remaining: 1m 4s


KeyboardInterrupt: 